# Imports

In [ ]:
%%capture

!pip install langchain
!pip install langchain-community
!pip install -U langchain-google-vertexai

!pip install google-api-core
!pip install google-auth
!pip install google-cloud-aiplatform
!pip install google-cloud-bigquery
!pip install google-cloud-bigquery-storage
!pip install google-cloud-core
!pip install google-cloud-resource-manager
!pip install google-cloud-storage
!pip install google-crc32c
!pip install google-resumable-media
!pip install googleapis-common-protos


In [ ]:
from langchain.prompts.chat import SystemMessagePromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.prompts.chat import AIMessagePromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain_google_vertexai import ChatVertexAI
from google.cloud import bigquery
import pandas as pd
import logging
import os

# initial Setup:

In [ ]:
SERVICE_ACCOUNT_CREDENTIALS = '/content/text-sql-project-ServiceAccountKey.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = SERVICE_ACCOUNT_CREDENTIALS

In [ ]:
DB_overview_file_path = '/content/Northwind_DB_Overview.txt'
schema_file_path = '/content/Nortwind_DB_Schema.txt'
relationships_file_path = '/content/Nortwind_DB_Relationships.txt'
tables_descriptions_file_path = '/content/Nortwind_DB_Table_Descriptions.txt'


with open(DB_overview_file_path, 'r') as file:
     db_overview = file.read()

with open(schema_file_path, 'r') as file:
     schema= file.read()

with open(relationships_file_path, 'r') as file:
     relationships = file.read()

with open(tables_descriptions_file_path, 'r') as file:
     table_fields_descriptions = file.read()



In [ ]:
schema

'Northwind Traders Database Schema:\n\n1. Products\nProductID: int, Primary Key, Not Null\nProductName: nvarchar(40), Not Null\nSupplierID: int, Foreign Key (Suppliers.SupplierID), Nullable\nCategoryID: int, Foreign Key (Categories.CategoryID), Nullable\nQuantityPerUnit: nvarchar(20), Nullable (Packaging details)\nUnitPrice: money, Nullable (Price per unit)\nUnitsInStock: smallint, Nullable (Current stock level)\nUnitsOnOrder: smallint, Nullable (Units currently on order)\nReorderLevel: smallint, Nullable (Threshold for reorder)\nDiscontinued: bit, Not Null (Whether the product is discontinued)\n\n\n2. Suppliers\nSupplierID: int, Primary Key, Not Null\nCompanyName: nvarchar(40), Not Null (Supplier’s company name)\nContactName: nvarchar(30), Nullable (Supplier’s contact person)\nContactTitle: nvarchar(30), Nullable (Title of the contact person)\nAddress: nvarchar(60), Nullable (Street address)\nCity: nvarchar(15), Nullable (City of the supplier)\nRegion: nvarchar(15), Nullable (Region o

In [ ]:
PROJECT = 'text-sql-project'

llm = ChatVertexAI(project = PROJECT,
                   model_name='gemini-1.5-flash-002',
                   temperature=0.0,
                   max_tokens=None,
                   max_retries=2)

In [ ]:
user_query = "What's the total sales amount for each month in 2023?"

# Text - to - SQL:

### STEP 1: Intent classification of user's text query:

In [ ]:
intent_examples = pd.read_csv('/content/prompts_and_intents.csv')
intent_examples[:10]

,Prompt,Intent
0,"Show me all orders placed by customer ""Alfreds...",RETRIEVE_ORDERS_BY_CUSTOMER_AND_DATE
1,"I need the list of orders from ""Around the Hor...",RETRIEVE_ORDERS_BY_CUSTOMER_AND_DATE
2,"Can you retrieve orders made by ""Berglunds sna...",RETRIEVE_ORDERS_BY_CUSTOMER_AND_DATE
3,"Fetch all orders for ""Blauer See Delikatessen""...",RETRIEVE_ORDERS_BY_CUSTOMER_AND_DATE
4,"Give me the orders from ""Bon app'"" in the last...",RETRIEVE_ORDERS_BY_CUSTOMER_AND_DATE
5,Who are our top 5 customers by total sales?,IDENTIFY_TOP_CUSTOMERS_BY_SALES
6,List the customers with the highest order valu...,IDENTIFY_TOP_CUSTOMERS_BY_SALES
7,Which customers have spent the most with us in...,IDENTIFY_TOP_CUSTOMERS_BY_SALES
8,Show me the top 10 customers in terms of total...,IDENTIFY_TOP_CUSTOMERS_BY_SALES
9,Find out which customers generated the most re...,IDENTIFY_TOP_CUSTOMERS_BY_SALES


In [ ]:
# Create prompt template and append system message, few shot examples and user query:
intent_messages = []

# System prompt:
template = "You are a helpful assistant capable of detecting the intent behind a user's query."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
intent_messages.append(system_message_prompt)


In [ ]:
# Adding intro th
few_shot_intro = "The following are few-shot examples of user's queries being intent classified. Use these as guidelines to classify the intent of the user's query."
system_message_few_shot_intro = SystemMessagePromptTemplate.from_template(few_shot_intro)
few_shot_tail = "x-----------------------------------x"
system_message_few_shot_tail = SystemMessagePromptTemplate.from_template(few_shot_tail)
intent_messages.append(system_message_few_shot_intro)
intent_messages



[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You are a helpful assistant capable of detecting the intent behind a user's query."), additional_kwargs={}),
 SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="The following are few-shot examples of user's queries being intent classified. Use these as guidelines to classify the intent of the user's query."), additional_kwargs={})]

In [ ]:
# Iterate through csv of few-shot examples to append to prompt template:
for _, row in intent_examples.iterrows():
    prompt, completion = row
    human_message = HumanMessagePromptTemplate.from_template(f"User: {prompt}")
    intent_messages.append(human_message)
    ai_message = AIMessagePromptTemplate.from_template(f"Assistant: {completion}")
    intent_messages.append(ai_message)
intent_messages.append(system_message_few_shot_tail)
intent_messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You are a helpful assistant capable of detecting the intent behind a user's query."), additional_kwargs={}),
 SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="The following are few-shot examples of user's queries being intent classified. Use these as guidelines to classify the intent of the user's query."), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='User: Show me all orders placed by customer "Alfreds Futterkiste" in September 2023.'), additional_kwargs={}),
 AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Assistant: RETRIEVE_ORDERS_BY_CUSTOMER_AND_DATE'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variab

In [ ]:
human_template = "User: {user_query}"
human_message = HumanMessagePromptTemplate.from_template(human_template)
intent_messages.append(human_message)

In [ ]:
intent_messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You are a helpful assistant capable of detecting the intent behind a user's query."), additional_kwargs={}),
 SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="The following are few-shot examples of user's queries being intent classified. Use these as guidelines to classify the intent of the user's query."), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='User: Show me all orders placed by customer "Alfreds Futterkiste" in September 2023.'), additional_kwargs={}),
 AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Assistant: RETRIEVE_ORDERS_BY_CUSTOMER_AND_DATE'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variab

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages(intent_messages)

# Format the prompt with the user's query and generate the message sequence
request = chat_prompt.format_prompt(user_query=user_query).to_messages()

In [ ]:
%%time

response = llm(request)
intent = response.content.strip()

CPU times: user 18.9 ms, sys: 4.29 ms, total: 23.2 ms
Wall time: 306 ms


In [ ]:
intent

'Assistant: CALCULATE_TOTAL_SALES_PER_MONTH'

### STEP 2: Entity extraction from the user's text query:

In [ ]:
NER_examples = pd.read_csv('/content/prompts_NER.csv')
NER_examples[:10]

,Prompt,Entities
0,Show me all orders shipped to Germany between ...,Ship Country: Germany | Start Date: January 1s...
1,List all products supplied by Exotic Liquids t...,Supplier Name: Exotic Liquids | Units In Stock...
2,Who are the customers from London who have pla...,Customer City: London | Order Date: Last month
3,Find employees who report to Andrew Fuller,Manager Name: Andrew Fuller | Relationship: Re...
4,Get me a list of all discontinued products,Discontinued Status: Yes (True)
5,Which products are in the Beverages category a...,Category Name: Beverages | Unit Price: Greater...
6,Show orders that were shipped via Speedy Expre...,Shipper Name: Speedy Express | Ship Date: Dece...
7,"List all employees hired after January 1st, 2020","Hire Date: After January 1st, 2020"
8,Which customers have not placed any orders in ...,Time Frame: Past year | Order Count: Zero (No ...
9,Find the top 5 most expensive products,Order By: Unit Price descending | Limit: Top 5


In [ ]:
entity_messages =[]

template = "You are a helpful assistant capable of performing named entity recognition."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
entity_messages.append(system_message_prompt)

In [ ]:
# Adding intro th
few_shot_intro = "The following are few-shot examples where entities are being extracted from the user's queries. Use these as guidelines to extract the entities from the user's query."
system_message_few_shot_intro = SystemMessagePromptTemplate.from_template(few_shot_intro)
few_shot_tail = "x-----------------------------------x"
system_message_few_shot_tail = SystemMessagePromptTemplate.from_template(few_shot_tail)
entity_messages.append(system_message_few_shot_intro)
entity_messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant capable of performing named entity recognition.'), additional_kwargs={}),
 SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="The following are few-shot examples where entities are being extracted from the user's queries. Use these as guidelines to extract the entities from the user's query."), additional_kwargs={})]

In [ ]:
# Iterate through csv of few-shot examples to append to prompt template:
for _, row in NER_examples.iterrows():
    prompt, completion = row
    human_message = HumanMessagePromptTemplate.from_template(f"User: {prompt}")
    entity_messages.append(human_message)
    ai_message = AIMessagePromptTemplate.from_template(f"Assistant: {completion}")
    entity_messages.append(ai_message)
entity_messages.append(system_message_few_shot_tail)
entity_messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant capable of performing named entity recognition.'), additional_kwargs={}),
 SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="The following are few-shot examples where entities are being extracted from the user's queries. Use these as guidelines to extract the entities from the user's query."), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='User: Show me all orders shipped to Germany between January 1st, 2023, and March 31st, 2023'), additional_kwargs={}),
 AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Assistant: Ship Country: Germany | Start Date: January 1st, 2023 | End Date: March 31st, 2023'), additional_kwargs={

In [ ]:
human_template = "User: {user_query}"
human_message = HumanMessagePromptTemplate.from_template(human_template)
entity_messages.append(human_message)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages(entity_messages)

# Format the prompt with the user's query and generate the message sequence
request = chat_prompt.format_prompt(user_query=user_query).to_messages()

In [ ]:
%%time

response = llm(request)
entities = response.content.strip()

CPU times: user 18.4 ms, sys: 3.03 ms, total: 21.5 ms
Wall time: 343 ms


In [ ]:
entities

'Assistant: Year: 2023 | Calculation: Total Sales Amount per Month'

### STEP 3: Mapping query and intent to relevant tables in the database:

In [ ]:
mapping_examples = pd.read_csv('/content/Intents_table_mapping.csv')
mapping_examples[:10]

,Query,Intent,Mapped_Tables
0,"Show me all orders placed by customer ""Alfreds...",RETRIEVE_ORDERS_BY_CUSTOMER_AND_DATE,Orders | Customers
1,Show me the top 10 customers in terms of total...,IDENTIFY_TOP_CUSTOMERS_BY_SALES,Orders | Order Details | Customers
2,Show me the total sales figures per month for ...,CALCULATE_TOTAL_SALES_PER_MONTH,Orders | Order Details
3,Which month had the greatest number of orders ...,FIND_PEAK_SHIPMENT_PERIODS,Orders
4,Provide a count of products grouped by their c...,GROUP_AND_COUNT_PRODUCTS_BY_CATEGORY,Products | Categories
5,List all items that have zero units in stock.,IDENTIFY_OUT_OF_STOCK_PRODUCTS,Products
6,Show me all employees located in London.,LIST_EMPLOYEES_BY_REGION,Employees | EmployeeTerritories | Territories ...
7,List all suppliers along with their contact na...,GET_SUPPLIER_INFORMATION,Suppliers
8,Find products marked as discontinued.,FIND_DISCONTINUED_PRODUCTS,Products
9,List products where units in stock are less th...,FIND_PRODUCTS_WITH_LOW_STOCK,Products


In [ ]:
# Create prompt template and append system message, few shot examples and user query:
mapping_messages = []

# System prompt:
template = "You are a helpful assistant capable of mapping detected intent to the correct list of tables from a database. You will be provided with a schema of the database and a detailed explanation of the databases tables and individual fields, which you must use to properly map the intent."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
mapping_messages.append(system_message_prompt)


In [ ]:
schema_intro = "Here is the schema of the database and descriptions of its tables and fields to help you understand the structure:"
system_message_schema_intro = SystemMessagePromptTemplate.from_template(schema_intro)
mapping_messages.append(system_message_schema_intro)

# Assuming 'schema' and 'table_fields_descriptions' contain the relevant details
system_message_schema = SystemMessagePromptTemplate.from_template(schema)
system_message_table_fields_descriptions = SystemMessagePromptTemplate.from_template(table_fields_descriptions)

# Append schema and table descriptions to messages
mapping_messages.append(system_message_schema)
mapping_messages.append(system_message_table_fields_descriptions)

In [ ]:
# Adding intro th
few_shot_intro = "The following are few-shot examples of intents being mapped to relavant tables. Use these as guidelines to map the intent to the relevant table(s)."
system_message_few_shot_intro = SystemMessagePromptTemplate.from_template(few_shot_intro)
few_shot_tail = "x-----------------------------------x"
system_message_few_shot_tail = SystemMessagePromptTemplate.from_template(few_shot_tail)
mapping_messages.append(system_message_few_shot_intro)
mapping_messages



[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant capable of mapping detected intent to the correct list of tables from a database. You will be provided with a schema of the database and a detailed explanation of the databases tables and individual fields, which you must use to properly map the intent.'), additional_kwargs={}),
 SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Here is the schema of the database and descriptions of its tables and fields to help you understand the structure:'), additional_kwargs={}),
 SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Northwind Traders Database Schema:\n\n1. Products\nProductID: int, Primary Key, Not Null\nProductName: nvarchar(40), Not Null\nSupplierID: int, Foreign Key (Suppliers.SupplierID), Nullable\nCate

In [ ]:
for _, row in mapping_examples.iterrows():
    row_query = row['Query ']
    row_intent = row['Intent']
    row_table_mapping = row['Mapped_Tables']

    # Format and append user message (Query and Intent)
    human_message = HumanMessagePromptTemplate.from_template(f"User: Query: {row_query}, Intent: {row_intent}")
    mapping_messages.append(human_message)

    # Format and append assistant message (table_mapping)
    ai_message = AIMessagePromptTemplate.from_template(f"Assistant: Mapped Tables: {row_table_mapping}")
    mapping_messages.append(ai_message)

# Append few-shot tail at the end
mapping_messages.append(system_message_few_shot_tail)
mapping_messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant capable of mapping detected intent to the correct list of tables from a database. You will be provided with a schema of the database and a detailed explanation of the databases tables and individual fields, which you must use to properly map the intent.'), additional_kwargs={}),
 SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Here is the schema of the database and descriptions of its tables and fields to help you understand the structure:'), additional_kwargs={}),
 SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Northwind Traders Database Schema:\n\n1. Products\nProductID: int, Primary Key, Not Null\nProductName: nvarchar(40), Not Null\nSupplierID: int, Foreign Key (Suppliers.SupplierID), Nullable\nCate

In [ ]:
human_template = "User: Query: {user_query}, Intent: {intent}"
human_message = HumanMessagePromptTemplate.from_template(human_template)
mapping_messages.append(human_message)


In [ ]:
chat_prompt = ChatPromptTemplate.from_messages(mapping_messages)

# Format the prompt with the user's query and generate the message sequence
request = chat_prompt.format_prompt(user_query=user_query, intent= intent ).to_messages()

In [ ]:
%%time

response = llm(request)
mapping = response.content.strip()

CPU times: user 17.1 ms, sys: 0 ns, total: 17.1 ms
Wall time: 350 ms


In [ ]:
mapping

'Assistant: Mapped Tables: Orders, Order Details'

### STEP 4: Creating SQL Query :

In [ ]:
query_creation_messages = []

template =  """
You are a SQL expert skilled in writing complex queries for BigQuery. You will be given a user's request in plain text, describing the data they need to extract from the database. Your task is to convert the user's text input into a valid SQL query to retrieve the relevant data from the database.

To help you construct accurate queries, you will be provided with:
- A detailed overview of the database, including its schema, relationships between the tables, and explanations of each table and its fields.
- The user's intent, extracted entities, and recommended tables based on their input.

"""

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
query_creation_messages.append(system_message_prompt)

In [ ]:
human_template = """Use this information to generate a correct SQL query that meets the user's requirements:

Northwind Traders Database Information:
--------------
Database Overview : {overview}\n
Database Schema : {schema}\n
Database Relationships : {relationships}
Detailed Descriptions of Tables and Fields : {descriptions}

User Query:
-----------
{user_query}


Input Parameters:
-----------------
INTENT: {intent}
EXTRACTED_ENTITIES: {entities}
RECOMMENDED_MAPPED_TABLES: {mapping}

NOTE : The database name in BigQuery is 'northwind_dataset'

"""

In [ ]:
human_message = HumanMessagePromptTemplate.from_template(human_template)
query_creation_messages.append(human_message)

chat_prompt = ChatPromptTemplate.from_messages(query_creation_messages)

request = chat_prompt.format_prompt(intent=intent,
                                    entities=entities,
                                    mapping=mapping,
                                    user_query=user_query,
                                    overview = db_overview,
                                    schema = schema,
                                    relationships = relationships,
                                    descriptions = table_fields_descriptions).to_messages()

In [ ]:
%%time

query_generation_response = llm(request)


CPU times: user 26.6 ms, sys: 1.14 ms, total: 27.7 ms
Wall time: 1.03 s


In [ ]:
llm_response = query_generation_response.content
filtered_query = llm_response.strip().replace("sql", "").replace("", "").strip()
filtered_query = filtered_query.replace("`","")
print(filtered_query)


SELECT
    strftime('%Y-%m', o.OrderDate) AS sales_month,
    SUM(od.Quantity * od.UnitPrice * (
      1 - od.Discount
    )) AS total_sales
  FROM
    northwind_dataset.Orders AS o
    INNER JOIN northwind_dataset.OrderDetails AS od ON o.OrderID = od.OrderID
  WHERE strftime('%Y', o.OrderDate) = '2023'
  GROUP BY 1
ORDER BY
  sales_month




In [ ]:
client = bigquery.Client()


query_job = client.query(filtered_query)

# Get the result and display it
results = query_job.result()

df= results.to_dataframe()
df

,CategoryName,ProductCount
0,Beverages,48
1,Condiments,48
2,Confections,52
3,Dairy Products,40
4,Grains/Cereals,28
5,Meat/Poultry,24
6,Produce,20
7,Seafood,48


In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [ ]:
import gradio as gr

def generate_query(user_input):
    query_creation_messages = []

    template =  """
    You are a SQL expert skilled in writing complex queries for BigQuery. You will be given a user's request in plain text, describing the data they need to extract from the database. Your task is to convert the user's text input into a valid SQL query to retrieve the relevant data from the database.

    To help you construct accurate queries, you will be provided with:
    - A detailed overview of the database, including its schema, relationships between the tables, and explanations of each table and its fields.
    - The user's intent, extracted entities, and recommended tables based on their input.

    """

    system_message_prompt = SystemMessagePromptTemplate.from_template(template)
    query_creation_messages.append(system_message_prompt)

    human_template = """Use this information to generate a correct SQL query that meets the user's requirements:

    Northwind Traders Database Information:
    --------------
    Database Overview : {overview}\n
    Database Schema : {schema}\n
    Database Relationships : {relationships}
    Detailed Descriptions of Tables and Fields : {descriptions}

    User Query:
    -----------
    {user_query}


    Input Parameters:
    -----------------
    INTENT: {intent}
    EXTRACTED_ENTITIES: {entities}
    RECOMMENDED_MAPPED_TABLES: {mapping}

    NOTE : The database name in BigQuery is 'northwind_dataset'

    """

    human_message = HumanMessagePromptTemplate.from_template(human_template)
    query_creation_messages.append(human_message)

    chat_prompt = ChatPromptTemplate.from_messages(query_creation_messages)

    request = chat_prompt.format_prompt(intent=intent,
                                        entities=entities,
                                        mapping=mapping,
                                        user_query=user_input,
                                        overview = db_overview,
                                        schema = schema,
                                        relationships = relationships,
                                        descriptions = table_fields_descriptions).to_messages()

    query_generation_response = llm(request)

    llm_response = query_generation_response.content
    filtered_query = llm_response.strip().replace("sql", "").replace("", "").strip()
    filtered_query = filtered_query.replace("`","")


    client = bigquery.Client()

    query_job = client.query(filtered_query)

    # Get the result and display it
    results = query_job.result()

    df= results.to_dataframe()

    return df



demo = gr.Interface(
    fn=generate_query,
    inputs=gr.Text(label="Enter Query", info="Please explain in plain text, what data you would like to retrieve"),
    outputs=[gr.DataFrame(df)],
)

if __name__ == "__main__":
    demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://94027ddaac29ca406e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
intent_examples

,Prompt,Intent
0,"Show me all orders placed by customer ""Alfreds...",RETRIEVE_ORDERS_BY_CUSTOMER_AND_DATE
1,"I need the list of orders from ""Around the Hor...",RETRIEVE_ORDERS_BY_CUSTOMER_AND_DATE
2,"Can you retrieve orders made by ""Berglunds sna...",RETRIEVE_ORDERS_BY_CUSTOMER_AND_DATE
3,"Fetch all orders for ""Blauer See Delikatessen""...",RETRIEVE_ORDERS_BY_CUSTOMER_AND_DATE
4,"Give me the orders from ""Bon app'"" in the last...",RETRIEVE_ORDERS_BY_CUSTOMER_AND_DATE
5,Who are our top 5 customers by total sales?,IDENTIFY_TOP_CUSTOMERS_BY_SALES
6,List the customers with the highest order valu...,IDENTIFY_TOP_CUSTOMERS_BY_SALES
7,Which customers have spent the most with us in...,IDENTIFY_TOP_CUSTOMERS_BY_SALES
8,Show me the top 10 customers in terms of total...,IDENTIFY_TOP_CUSTOMERS_BY_SALES
9,Find out which customers generated the most re...,IDENTIFY_TOP_CUSTOMERS_BY_SALES
